In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.decomposition import PCA
from sklearn.ensemble import IsolationForest

# constants
pca_components = 8  # I used 8 for a dataset with 14 features

In [10]:
df = pd.read_csv("ml_models_dataset_with_zone.csv", index_col='timestamp')
df.head()

,cell_name,ho_failure_rate,num_voice_attempts,voice_drop_rate,num_data_attempts,voice_setup_failure_rate,voice_tot_failure_rate,avail_period_duration,bandwidth,throughput_rate,...,tech_freq_W,tech_freq_X,tech_freq_Y,tech_freq_Z,zone_-1,zone_0,zone_1,zone_2,zone_3,zone_4
timestamp,,,,,,,,,,,,,,,,,,,,,
2019-11-04 00:00:00+00:00,22_21Q,0.000679,0.000679,0.000679,0.000679,0.000679,0.000679,1.0,0.09955,6.786382e-04,...,0,0,0,0,0,0,1,0,0,0
2019-11-04 00:00:00+00:00,02_31Q,0.006013,0.000000,0.006013,0.000781,0.006013,0.006013,1.0,0.09955,2.322707e-08,...,0,0,0,0,0,0,0,1,0,0
2019-11-04 00:00:00+00:00,25_11W,0.001574,0.001574,0.001574,0.001574,0.001574,0.001574,1.0,1.00000,1.573807e-03,...,1,0,0,0,0,0,1,0,0,0
2019-11-04 00:00:00+00:00,28_11Y,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.00000,0.000000e+00,...,0,0,1,0,0,0,0,1,0,0
2019-11-04 00:00:00+00:00,12_21W,0.001032,0.013902,0.000000,0.091727,0.000000,0.000000,1.0,1.00000,5.298183e-04,...,1,0,0,0,0,0,1,0,0,0


In [11]:
# Help functions

def get_cell_df(cell_name):
  return df[df['cell_name'] == cell_name]

def add_aggregated_time_information(data, window_size=5):
  time = np.zeros(shape=data.shape)
  for i in range(window_size, time.shape[0]):
    time[i] = np.mean(data[i-window_size:i], axis=0)

  return np.concatenate((data, time), axis=1)

In [48]:
def getIsolationForestScore(df, pca_components):
    """
    df: pandas dataframe, preprocessed (only numerical values)
    pca_components: int, number of components to reduce to with pca

    returns: list of floats, score for each example in df
    """
    pca = PCA(n_components=pca_components)
    data = pca.fit_transform(df)
    data = add_aggregated_time_information(data)
    model =  IsolationForest(contamination = 0.1, random_state=42)
    model.fit(data)
    return model.score_samples(data)

def runIsolationForestForAllCells(df, pca_components):
    """
    df: pandas dataframe, index: timestamp - cell_name must be included - preprocessed
    """
    df['anom_scores'] = np.zeros(df.shape[0])
    for i, cell in enumerate(df['cell_name'].unique()[:5]):
        # Hardcode dropping of cells with very little variation
        if cell == '29_11Q':
            continue
        if cell == '28_21Y':
            continue
        
        # Get df with only the current cell_name-rows
        df_temp = get_cell_df(cell)
        df_temp = df_temp.drop(columns='cell_name')
        # Reduce dimentions, fit iForest
        scores = getIsolationForestScore(df_temp, pca_components)
        # Add scores to original df
        df.loc[df['cell_name'] == cell, 'anom_scores'] = scores
        print(df)
        if i%50 == 0:
            print('.... done with:', i)
    return df

In [49]:
df_with_scores = runIsolationForestForAllCells(df, pca_components)

                          cell_name  ho_failure_rate  num_voice_attempts  \
timestamp                                                                  
2019-11-04 00:00:00+00:00    22_21Q         0.000679            0.000679   
2019-11-04 00:00:00+00:00    02_31Q         0.006013            0.000000   
2019-11-04 00:00:00+00:00    25_11W         0.001574            0.001574   
2019-11-04 00:00:00+00:00    28_11Y         0.000000            0.000000   
2019-11-04 00:00:00+00:00    12_21W         0.001032            0.013902   
...                             ...              ...                 ...   
2020-12-21 23:00:00+00:00    00_11Y         0.000000            0.005561   
2020-12-21 23:00:00+00:00    00_31R         0.333333            0.001854   
2020-12-21 23:00:00+00:00    03_31Q         0.000339            0.000000   
2020-12-21 23:00:00+00:00    13_31Q         0.000152            0.000000   
2020-12-21 23:00:00+00:00    21_11X         0.333333            0.002780   

           

                          cell_name  ho_failure_rate  num_voice_attempts  \
timestamp                                                                  
2019-11-04 00:00:00+00:00    22_21Q         0.000679            0.000679   
2019-11-04 00:00:00+00:00    02_31Q         0.006013            0.000000   
2019-11-04 00:00:00+00:00    25_11W         0.001574            0.001574   
2019-11-04 00:00:00+00:00    28_11Y         0.000000            0.000000   
2019-11-04 00:00:00+00:00    12_21W         0.001032            0.013902   
...                             ...              ...                 ...   
2020-12-21 23:00:00+00:00    00_11Y         0.000000            0.005561   
2020-12-21 23:00:00+00:00    00_31R         0.333333            0.001854   
2020-12-21 23:00:00+00:00    03_31Q         0.000339            0.000000   
2020-12-21 23:00:00+00:00    13_31Q         0.000152            0.000000   
2020-12-21 23:00:00+00:00    21_11X         0.333333            0.002780   

           

                          cell_name  ho_failure_rate  num_voice_attempts  \
timestamp                                                                  
2019-11-04 00:00:00+00:00    22_21Q         0.000679            0.000679   
2019-11-04 00:00:00+00:00    02_31Q         0.006013            0.000000   
2019-11-04 00:00:00+00:00    25_11W         0.001574            0.001574   
2019-11-04 00:00:00+00:00    28_11Y         0.000000            0.000000   
2019-11-04 00:00:00+00:00    12_21W         0.001032            0.013902   
...                             ...              ...                 ...   
2020-12-21 23:00:00+00:00    00_11Y         0.000000            0.005561   
2020-12-21 23:00:00+00:00    00_31R         0.333333            0.001854   
2020-12-21 23:00:00+00:00    03_31Q         0.000339            0.000000   
2020-12-21 23:00:00+00:00    13_31Q         0.000152            0.000000   
2020-12-21 23:00:00+00:00    21_11X         0.333333            0.002780   

           

In [52]:
df_with_scores.describe()

,ho_failure_rate,num_voice_attempts,voice_drop_rate,num_data_attempts,voice_setup_failure_rate,voice_tot_failure_rate,avail_period_duration,bandwidth,throughput_rate,data_setup_failure_rate,...,tech_freq_X,tech_freq_Y,tech_freq_Z,zone_-1,zone_0,zone_1,zone_2,zone_3,zone_4,anom_scores
count,1.042688e+06,1.042688e+06,1.042688e+06,1.042688e+06,1.042688e+06,1.042688e+06,1.042688e+06,1.042688e+06,1.042688e+06,1.042688e+06,...,1.042688e+06,1.042688e+06,1.042688e+06,1.042688e+06,1.042688e+06,1.042688e+06,1.042688e+06,1.042688e+06,1.042688e+06,1.042688e+06
mean,1.533228e-01,1.087738e-02,7.895436e-04,1.824066e-02,9.470607e-04,9.869756e-04,9.990099e-01,3.435080e-01,2.523121e-04,2.379716e-03,...,8.109041e-02,7.512698e-02,1.261844e-01,1.256368e-04,2.540913e-01,3.129258e-01,1.979298e-01,1.379943e-01,9.693312e-02,-5.023199e-03
std,1.713096e-01,2.641081e-02,8.046650e-03,3.614324e-02,1.473337e-02,9.447519e-03,2.345380e-02,3.452026e-01,1.820752e-03,1.841730e-02,...,2.729740e-01,2.635963e-01,3.320573e-01,1.120808e-02,4.353494e-01,4.636847e-01,3.984391e-01,3.448942e-01,2.958668e-01,4.527398e-02
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-8.721487e-01
25%,3.194319e-05,0.000000e+00,0.000000e+00,1.442963e-03,0.000000e+00,0.000000e+00,1.000000e+00,9.954977e-02,1.746050e-06,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,1.573807e-03,1.853568e-03,0.000000e+00,5.785091e-03,0.000000e+00,0.000000e+00,1.000000e+00,9.954977e-02,8.982702e-05,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,3.333333e-01,9.267841e-03,9.856101e-05,1.891738e-02,5.506305e-05,1.727429e-04,1.000000e+00,4.997499e-01,2.741481e-04,1.250855e-03,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00


In [ ]:
#cell_names_to_plot = ['00_11Z', '22_21Q', '13_21X'] # Not 29_11Q or 28_21Y
cell_name_to_plot = df['cell_name'].unique()

cell_names_to_plots = [cell for cell in cell_name_to_plot if cell != '29_11Q']
cell_names_to_plots2 = [cell for cell in cell_names_to_plots if cell != '28_21Y']

for cell_to_plot in cell_names_to_plots2:
    
  plt.scatter(df_with_scores[df_with_scores['cell_name'] == cell_to_plot].index, df_with_scores[df_with_scores['cell_name'] == cell_to_plot]['anom_scores'])

plt.show()